In [1]:
import nbimporter
import numpy as np
import matplotlib.pyplot as plt
import time

import gas_water
import liquid_water
import solid_material

import flux
import balance_equation
import property_conversion  as prop
import moisture_conversion as mc

Importing Jupyter notebook from gas_water.ipynb
Importing Jupyter notebook from moisture_conversion.ipynb
Importing Jupyter notebook from vapour_pressure.ipynb
Importing Jupyter notebook from liquid_water.ipynb
Importing Jupyter notebook from property_conversion.ipynb
Importing Jupyter notebook from solid_material.ipynb
Importing Jupyter notebook from flux.ipynb
Importing Jupyter notebook from balance_equation.ipynb


## 1次元モデルの差分計算をするためのモジュール  


### クラス：Cell( i )  
差分法におけるコントロールボリューム（CV）に相当する構造体。  
固相・液相・気相の情報から構成され、CVの大きさ、位置などの情報を有する。  
示量変数はCellクラスに内包され、示強変数はそれぞれの相情報に保持される。

- cell情報  
    i：セルの位置    
    dx：セルの大きさ    
    
    - 熱・水分・物質移動係数
        各種移動係数は後述のcellの構成情報に影響を受けるためcellクラスが保有する

- 気相情報(Gas)  
    - 水蒸気(gas_water)  
        temp：温度[K]  
        rh：相対湿度[-]  
        miu：水分化学ポテンシャル[J/kg]  
        pv：水蒸気圧[Pa]  
    - 空気（未実装）


- 液相情報(Liquid)  
    - 純水(liquid_water)  
        temp：温度[K]  
        miu：水分化学ポテンシャル[J/kg]  
        phi：含水率[-]  
        pl：水圧[Pa]  
    - 塩溶液（未実装）  
        

- 固相情報(Solid)  
    - 材料骨格部(solid_material)  
        material：材料の名称  
        crow：熱容量（比熱×密度）  
        ※以下は液相・気相の影響を受ける  
        lam：熱伝達率  
        phi：含水率  
        dphi：含水率の水分化学ポテンシャル微分  
        ldp：透湿率  
        ldml：水分化学ポテンシャル勾配に対する液相水分伝導率  
    - 氷（未実装）   
    - 塩（未実装）  


In [2]:
class  Cell():
    def __init__( self, i ):
        self.position = i
    
    # cell information
    # Length of cell
    def set_dx( self, initial = 0.0 ):
        self.dx = initial 
    
    # Gas condition（気相）
    def set_Gas_Water_BasedMiu( self, temp = 0.0, miu = 0.0 ):
        self.vapour = gas_water.MiuBasedMoisture( temp, miu )
        
    def set_Gas_Water_BasedRH( self, temp = 0.0, rh = 0.0 ):
        self.vapour = gas_water.RHBasedMoisture( temp, rh )
        
    def set_Gas_Water_BasedPV( self, temp = 0.0, pv = 0.0 ):
        self.vapour = gas_water.PVBasedMoisture( temp, pv )
        
    # Liquid condition（液相）
    def set_Liquid_Water_BasedMiu( self, temp = 0.0, miu = 0.0 ):
        self.water = liquid_water.MiuBasedWater( temp, miu )
        
    def set_Liquid_Water_BasedPhi( self, temp = 0.0, phi = 0.0 ):
        self.water = liquid_water.PhiBasedWater( temp, phi )
        
    # Solid condition（固相）
    def set_Solid_Material( self, temp, material_name ):
        self.material = solid_material.TempBased( temp, material_name )
        
    # Transfer coefficient（セルの有する移動係数）
    def set_TransferCoefficient( self ):
        self.lam = prop.Kernel_TransferCoefficient( self.material.name ).getLAM( self.water )
        self.ldml= prop.Kernel_TransferCoefficient( self.material.name ).getLDML( self.water )
        self.dp  = prop.Kernel_TransferCoefficient( self.material.name ).getDP( self.water )
        

### 使用例

セルの個数を決定する

In [3]:
number = 5

セルの作成・大きさを与える

In [4]:
c = [ Cell(i) for i in range(number)]
[ c[i].set_dx(0.01*i) for i in range(number)]

[None, None, None, None, None]

水蒸気の情報（温度20℃、相対湿度60%）を与える

In [5]:
[ c[i].set_Gas_Water_BasedRH( temp = 293.15+i, rh = 0.6 ) for i in range(len(c)) ]

[None, None, None, None, None]

材料情報を与える

In [6]:
[ c[i].set_Solid_Material( temp = c[i].vapour.temp, material_name = 'BentheimerSandstone' ) for i in range(len(c))]

[None, None, None, None, None]

水蒸気・材料の情報から液水の情報を算出する

In [7]:
[ c[i].set_Liquid_Water_BasedMiu( temp = c[i].vapour.temp, miu = c[i].vapour.miu ) for i in range(len(c)) ]
[ c[i].water.set_material( material_name = c[i].material.name ) for i in range(len(c)) ]

[None, None, None, None, None]

セルが持つ各種移動係数を計算する

In [8]:
[ c[i].set_TransferCoefficient() for i in range(len(c)) ]

Importing Jupyter notebook from bentheimer_sandstone.ipynb
Importing Jupyter notebook from van_genuchten.ipynb


[None, None, None, None, None]

#### 値の確認
セル情報の構築は以上となる。セル情報をいくつか引き出しみる。  
例えば液水の含水率は

In [9]:
c[1].water.phi

3.249797297402751e-05

セルの液相水分伝導率は

In [10]:
c[1].ldml

2.415612652351753e-23

### クラス：Air( i )  
材料境界などに位置する室空気のクラス。  
Cellクラスと基本概念は同じであるが固相・液相情報は持たず気相情報のみを持つものとする。  
また、Cellクラス同様に空間の容積などの示量変数はAirクラスに内包され、示強変数はそれぞれの相情報に保持される。  
現状、室空気は温度と水分情報しか持たないことが多いため、gas_waterのクラスのみを用いることで代用した。

- Air情報     
    i : Airの名称（基本的には何でもよい）
    vol：Airの容積[m3]    
    
    - 熱・水分・物質伝達係数\

- 気相情報(Gas)  
    - 水蒸気(gas_water)  
        temp：温度[K]  
        rh：相対湿度[-]  
        miu：水分化学ポテンシャル[J/kg]  
        pv：水蒸気圧[Pa]  
    - 空気（未実装）


In [11]:
class  Air():
    def __init__( self, i ):
        self.position = i
    
    # cell information
    # Length of cell
    def set_volume( self, volume = 0.0 ):
        self.vol = volume 
    
    # Gas condition（気相）
    def set_Air_BasedMiu( self, temp = 0.0, miu = 0.0 ):
        self.air = gas_water.MiuBasedMoisture( temp, miu )
        
    def set_Air_BasedRH( self, temp = 0.0, rh = 0.0 ):
        self.air = gas_water.RHBasedMoisture( temp, rh )
        
    def set_Air_BasedPV( self, temp = 0.0, pv = 0.0 ):
        self.air = gas_water.PVBasedMoisture( temp, pv )
        
    # Transfer coefficient（セルの有する移動係数）
    def set_HeatTransferCoefficient( self, alpha ):
        self.alpha = alpha
        
    def set_MoistureTransferCoefficient( self, aldm ):
        self.aldm = aldm
        

### 使用例
基本はCellクラスと同じなので

In [12]:
# 使用例
a = Air('室空気')

In [13]:
a.set_volume(100.0)
a.set_Air_BasedRH( temp = 283.15, rh = 0.8 ) 
a.set_HeatTransferCoefficient( alpha = 9.3 )
a.set_MoistureTransferCoefficient( aldm = 9.3e-10 )

#### 値の確認

In [14]:
a.air.pv

982.3962203526237

## 流量計算  

Fluxモジュールは2つのCellを与えることでその間の熱・水分流を計算する。  
例えばcell[1]とcell[2]間の流量は、

In [15]:
flux1 = flux.HeatAndMoisture_VapourFluxBasedPv( cell_mns = c[1], cell_pls = c[2], nx =0.0 )

熱流の計算結果は、

In [16]:
flux1.qs

-80.0

水蒸気流の計算結果は、

In [17]:
flux1.jv

-1.2565291386843607e-06

液水流の計算結果は、

In [18]:
flux1.jl

3.7586490858133714e-19

同様に空気層との流量計算もでき、

In [19]:
flux2 = flux.HeatAndMoisture_VapourFluxBasedPv( cell_mns = a, cell_pls = c[0], nx =0.0 )

熱流

In [20]:
flux2.qs

-93.0

水蒸気流

In [21]:
flux2.jv

-3.914239797133416e-07

In [22]:
flux2.jl

0.0

これを一括計算しようとすると以下のようになる。  
例えば  

空気　―　壁（cell[0] ～　cell[end]）　―　空気  

のような状況を想定する。  

※wall = c とするとmutableなオブジェクトとしてコピーされるのでスライスで対処

In [23]:
wall1 = c[:]
wall1.insert(0,a)
wall1.insert(len(wall1),a)

wall1のコンストラクタを確認してみる。まずは空気層のある場所について、

In [24]:
[ hasattr(wall1[i], 'air') for i in range(len(wall1)) ]

[True, False, False, False, False, False, True]

次に多孔質材料の有無、

In [25]:
[ hasattr(wall1[i], 'material') for i in range(len(wall1)) ]

[False, True, True, True, True, True, False]


流量を計算してみる。

In [26]:
flux3 = [ flux.HeatAndMoisture_VapourFluxBasedPv( cell_mns = wall1[i], cell_pls = wall1[i+1], nx =0.0 ) for i in range(len(wall1)-1) ]

#### 値の確認  
熱流

In [27]:
[ flux3[i].qs for i in range(len(flux3)) ]

[-93.0,
 -240.0,
 -80.0,
 -47.99999999999999,
 -34.285714285714285,
 112.72727272727273]

水蒸気流

In [28]:
[ flux3[i].jv for i in range(len(flux3)) ]

[-3.914239797133416e-07,
 -3.5722675755550987e-06,
 -1.2565291386843607e-06,
 -7.952053338607731e-07,
 -5.988450050732532e-07,
 6.880735248531062e-07]

液相水分流

In [29]:
[ flux3[i].jl for i in range(len(flux3)) ]

[0.0,
 1.1391635302763927e-18,
 3.7586490858133714e-19,
 2.2233362044433623e-19,
 1.5648339625526345e-19,
 0.0]

## 熱・水分の収支計算

### 値を取得するクラス  
#### クラス : logger()  
ある点の温度をlogging間隔ごとに取得するクラス  

#### クラス：CrossSection()  
ある時刻における全セルの状態を取得するクラス  

In [30]:
class CrossSection():
    def __init__( self, interval ):
        self.interval = interval
        self.times = []
        self.cell  = []
        
    def writeData( self, t, cell, dt):
        if t % self.interval == 0:
            self.times.append( t * dt)
            self.cell.append(cell)

### Draw Graph（グラフの描画をする関数）  

In [31]:
# draw graph
def drawGraph( cell, time ):
    temp = [ cell[i].temp - 273.15 for i in range(L) ]
    dx   = [ cell[i].position*cell[i].dx for i in range(L) ]

    fig = plt.figure(figsize=(15,4))
    ax = fig.add_subplot(1,1,1)
    ax.plot( dx, temp, label = time )
    ax.set_xlabel( 'xlength[m]', fontsize = 12 )
    ax.set_ylabel( 'temp[K]', fontsize = 12 )
    plt.ylim([10.0, 20.0])
    plt.legend()
    plt.show()